## 4. Regression Analysis 

In [1]:
import cmocean
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import statsmodels.api as sm

from feature_engineering import feature_engineer

%matplotlib inline

# Read data from the data folder
file_directory = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '')) + '\\'
horse_race_df = pd.read_csv(file_directory + 'data/horse_race.csv', low_memory=False, index_col=0)
horse_race_df['age_int'] = horse_race_df['sex_age'].apply(lambda x: re.search(r'\d+', x).group(0)).astype(int)

# Do some simple data transformation
horse_race_df['run_date'] = horse_race_df['run_date'].apply(pd.Timestamp)
horse_race_df = horse_race_df.sort_values(['horse_id', 'run_date'])
try:
    first_occur_df = pd.read_csv(file_directory + 'data/first_occurence_race.csv', low_memory=False, index_col=0)
    first_occur_df['run_date'] = first_occur_df['run_date'].apply(pd.Timestamp)
    first_occur_df = first_occur_df.sort_values(['horse_id', 'run_date'])
except FileNotFoundError:
    horse_race_sorted = horse_race_df.copy()
    horse_id_set = set()
    first_occur_dict = {}
    for index, value in horse_race_sorted.iterrows():
        if value['horse_id'] not in horse_id_set:
            horse_id_set.add(value['horse_id'])
            first_occur_dict[index] = value
    first_occur_df = pd.DataFrame.from_dict(first_occur_dict, orient='index')
    first_occur_df.to_csv(file_directory + 'data/first_occurence_race.csv', encoding='utf-8')
    
columns_to_drop = [
    'race', 'title', 'horse', 'sex_age',
    'distance', 'run_time', 'breeder',
    'jockey', 'margin', 'trainer_x', 'trainer_y', 'owner_x', 'owner_y', 'horse_name', 'date_of_birth', 
    'transaction_price', 'prize_obtained', 'race_record', 'highlight_race', 'relatives', 'status', 'prize'
]
for column in columns_to_drop:
    try:
        first_occur_df.drop(column, axis=1, inplace=True)
        horse_race_df.drop(column, axis=1, inplace=True)
    except ValueError:
        continue
        
horse_race_df = horse_race_df[horse_race_df['finishing_position'].apply(lambda x: bool(re.search(r'\d+', x)))]
horse_race_df['finishing_position'] = horse_race_df['finishing_position'].apply(lambda x: re.search(r'\d+', x).group(0))
horse_race_df['finishing_position'] = horse_race_df['finishing_position'].astype(int)

C:\Users\terry\Anaconda3\lib\site-packages\cmocean\tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):
C:\Users\terry\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## 4.1 Feature Engineering 

In [2]:
# Functions have been moved to feature_engineering.py for code reuse
# See feature_engineering.py for more function details

if 1 == 0:
    # TODO: Check parent information (70%+ missing values)
    columns_to_drop_again = ['run_date', 'horse_id', 'parents']
    new_df = first_occur_df[columns_to_drop_again].copy()

    target_columns = ['horse_id', 'date_of_birth', 'breeder_id', 'gender', 'breed', 'race_record']
    new_df['parent_id_1'] = new_df['parents'].apply(lambda x: x.split(' ')[0])
    new_df['parent_id_2'] = new_df['parents'].apply(lambda x: x.split(' ')[1])
    new_df.drop('parents', axis=1, inplace=True)
    new_df = new_df[new_df['parent_id_1'].isin(horse_df['horse_id'].astype(str))]
    new_df = new_df[new_df['parent_id_2'].isin(horse_df['horse_id'].astype(str))]

    new_df.tail()

## 4.2 Regression Analysis

### 4.2.1 OLS for First Occurence Race

In [3]:
new_df_first = feature_engineer(first_occur_df)
X_first = new_df_first.loc[:, new_df_first.columns != 'run_time_1000']
y_first = new_df_first.loc[:, 'run_time_1000']
X_first = sm.add_constant(X_first)
results = sm.OLS(y_first, X_first).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          run_time_1000   R-squared:                       0.433
Model:                            OLS   Adj. R-squared:                  0.431
Method:                 Least Squares   F-statistic:                     247.5
Date:                Tue, 20 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:38:24   Log-Likelihood:            -1.3522e+05
No. Observations:               67877   AIC:                         2.709e+05
Df Residuals:                   67667   BIC:                         2.728e+05
Df Model:                         209                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                 27.6812      5.030      5.503      0.000      17.821      37.541
jockey_weight                          0.0788      0.008      9.787      0.000       0.063       0.095
win_odds                               0.0003      0.000      2.069      0.039    1.32e-05       0.000
win_fav                                0.1154      0.003     41.803      0.000       0.110       0.121
horse_weight                           0.0072      0.000     28.365      0.000       0.007       0.008
curr_age                              -0.6282      0.026    -24.384      0.000      -0.679      -0.578
horse_weight_increase                 -0.0492      0.002    -20.738      0.000      -0.054      -0.045
age_stated                             0.3200      0.017     18.387      0.000       0.286       0.354
jockey_age                            -0.0040      0.001     -3.499      0.000      -0.006      -0.002
trainer_age                           -0.0018      0.001     -1.818      0.069      -0.004       0.000
year_minus_one                         0.0163      0.003      6.499      0.000       0.011       0.021
first_last_year_jockey                -0.0128      0.009     -1.366      0.172      -0.031       0.006
second_last_year_jockey                0.0018      0.007      0.267      0.790      -0.011       0.015
third_last_year_jockey                 0.0024      0.007      0.358      0.721      -0.011       0.015
out_last_year_jockey                   0.0034      0.006      0.530      0.596      -0.009       0.016
races_major_last_year_jockey           0.0029      0.007      0.439      0.661      -0.010       0.016
wins_major_last_year_jockey           -0.0063      0.005     -1.283      0.199      -0.016       0.003
races_special_last_year_jockey        -0.0017      0.007     -0.266      0.790      -0.015       0.011
wins_special_last_year_jockey         -0.0049      0.004     -1.269      0.204      -0.012       0.003
races_flat_last_year_jockey           -0.0008      0.007     -0.115      0.909      -0.014       0.012
wins_flat_last_year_jockey            -0.0016      0.004     -0.459      0.646      -0.009       0.005
races_grass_last_year_jockey          -0.0027      0.002     -1.581      0.114      -0.006       0.001
wins_grass_last_year_jockey            0.0204      0.010      2.013      0.044       0.001       0.040
races_dirt_last_year_jockey           -0.0024      0.002     -1.411      0.158      -0.006       0.001
wins_dirt_last_year_jockey             0.0186      0.010      1.845      0.065      -0.001       0.038
wins_percent_last_year_jockey         -0.8626      0.732     -1.178      0.239      -2.298       0.573
wins_percent_2nd_last_year_jockey      0.7289      0.698      1.044      0.296      -0.639      

### 4.2.2 OLS for Full Race

In [4]:
new_df_full = feature_engineer(horse_race_df)
X_full = new_df_full.loc[:, new_df_full.columns != 'run_time_1000']
y_full = new_df_full.loc[:, 'run_time_1000']
X_full = sm.add_constant(X_full)
results = sm.OLS(y_full, X_full).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          run_time_1000   R-squared:                       0.572
Model:                            OLS   Adj. R-squared:                  0.572
Method:                 Least Squares   F-statistic:                     4340.
Date:                Tue, 20 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:40:37   Log-Likelihood:            -1.3005e+06
No. Observations:              677627   AIC:                         2.601e+06
Df Residuals:                  677417   BIC:                         2.604e+06
Df Model:                         209                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                 68.8892      1.099     62.677      0.000      66.735      71.043
jockey_weight                          0.0205      0.002     11.203      0.000       0.017       0.024
win_odds                               0.0014   3.23e-05     43.404      0.000       0.001       0.001
win_fav                                0.0710      0.001     96.891      0.000       0.070       0.072
horse_weight                           0.0019   7.85e-05     23.941      0.000       0.002       0.002
curr_age                              -0.5944      0.006   -100.763      0.000      -0.606      -0.583
horse_weight_increase                 -0.0092      0.000    -21.543      0.000      -0.010      -0.008
age_stated                             0.2870      0.006     51.122      0.000       0.276       0.298
jockey_age                            -0.0011      0.000     -3.460      0.001      -0.002      -0.000
trainer_age                           -0.0037      0.000    -12.264      0.000      -0.004      -0.003
year_minus_one                        -0.0164      0.001    -22.487      0.000      -0.018      -0.015
first_last_year_jockey                -0.0078      0.003     -2.891      0.004      -0.013      -0.003
second_last_year_jockey               -0.0032      0.002     -1.607      0.108      -0.007       0.001
third_last_year_jockey                -0.0023      0.002     -1.155      0.248      -0.006       0.002
out_last_year_jockey                  -0.0028      0.002     -1.422      0.155      -0.007       0.001
races_major_last_year_jockey           0.0032      0.002      1.612      0.107      -0.001       0.007
wins_major_last_year_jockey           -0.0029      0.001     -1.967      0.049      -0.006   -1.06e-05
races_special_last_year_jockey         0.0035      0.002      1.745      0.081      -0.000       0.007
wins_special_last_year_jockey         -0.0021      0.001     -1.889      0.059      -0.004    7.99e-05
races_flat_last_year_jockey            0.0039      0.002      1.979      0.048    3.73e-05       0.008
wins_flat_last_year_jockey            -0.0028      0.001     -2.710      0.007      -0.005      -0.001
races_grass_last_year_jockey          -0.0008      0.000     -1.848      0.065      -0.002    4.89e-05
wins_grass_last_year_jockey            0.0074      0.003      2.627      0.009       0.002       0.013
races_dirt_last_year_jockey           -0.0013      0.000     -3.062      0.002      -0.002      -0.000
wins_dirt_last_year_jockey             0.0086      0.003      3.085      0.002       0.003       0.014
wins_percent_last_year_jockey          0.0963      0.224      0.429      0.668      -0.343       0.536
wins_percent_2nd_last_year_jockey      0.2979      0.241      1.235      0.217      -0.175      

In [5]:
horse_race_df_grp_by = horse_race_df.set_index(['horse_id', 'run_date'])
horse_race_df_grp_by['run_time_diff'] = horse_race_df_grp_by['run_time_1000'].diff()
horse_race_df_grp_by = horse_race_df_grp_by[~horse_race_df_grp_by.index.isin(first_occur_df.set_index(['horse_id', 
                                                                                                       'run_date']).index)]
horse_race_df_grp_by.reset_index(inplace=True)
new_df_full_diff = feature_engineer(horse_race_df_grp_by)
new_df_full_diff['last_run_time'] = new_df_full_diff['run_time_1000'] - new_df_full_diff['run_time_diff']
new_df_full_diff.drop('run_time_diff', inplace=True, axis=1)

X_full_diff = new_df_full_diff.loc[:, new_df_full_diff.columns != 'run_time_1000']
y_full_diff = new_df_full_diff.loc[:, 'run_time_1000']
X_full_diff = sm.add_constant(X_full_diff)
results_diff = sm.OLS(y_full_diff, X_full_diff).fit()
results_diff.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          run_time_1000   R-squared:                       0.648
Model:                            OLS   Adj. R-squared:                  0.648
Method:                 Least Squares   F-statistic:                     5348.
Date:                Tue, 20 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:42:40   Log-Likelihood:            -1.1134e+06
No. Observations:              609756   AIC:                         2.227e+06
Df Residuals:                  609545   BIC:                         2.230e+06
Df Model:                         210                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                 52.6417      1.061     49.614      0.000      50.562      54.721
jockey_weight                          0.0085      0.002      4.892      0.000       0.005       0.012
win_odds                               0.0009   3.09e-05     29.304      0.000       0.001       0.001
win_fav                                0.0404      0.001     57.334      0.000       0.039       0.042
horse_weight                          -0.0002   7.62e-05     -2.559      0.010      -0.000   -4.57e-05
curr_age                              -0.2941      0.006    -50.248      0.000      -0.306      -0.283
horse_weight_increase                  0.0020      0.000      4.749      0.000       0.001       0.003
age_stated                             0.1306      0.006     23.332      0.000       0.120       0.142
jockey_age                            -0.0006      0.000     -2.035      0.042      -0.001   -2.37e-05
trainer_age                           -0.0022      0.000     -7.349      0.000      -0.003      -0.002
year_minus_one                        -0.0143      0.001    -20.316      0.000      -0.016      -0.013
first_last_year_jockey                -0.0051      0.003     -1.955      0.051      -0.010    1.24e-05
second_last_year_jockey               -0.0025      0.002     -1.279      0.201      -0.006       0.001
third_last_year_jockey                -0.0014      0.002     -0.749      0.454      -0.005       0.002
out_last_year_jockey                  -0.0022      0.002     -1.152      0.249      -0.006       0.002
races_major_last_year_jockey           0.0015      0.002      0.778      0.437      -0.002       0.005
wins_major_last_year_jockey           -0.0024      0.001     -1.708      0.088      -0.005       0.000
races_special_last_year_jockey         0.0026      0.002      1.381      0.167      -0.001       0.006
wins_special_last_year_jockey         -0.0008      0.001     -0.753      0.452      -0.003       0.001
races_flat_last_year_jockey            0.0027      0.002      1.392      0.164      -0.001       0.006
wins_flat_last_year_jockey            -0.0019      0.001     -1.863      0.062      -0.004    9.71e-05
races_grass_last_year_jockey          -0.0002      0.000     -0.510      0.610      -0.001       0.001
wins_grass_last_year_jockey            0.0040      0.003      1.476      0.140      -0.001       0.009
races_dirt_last_year_jockey           -0.0008      0.000     -2.024      0.043      -0.002   -2.66e-05
wins_dirt_last_year_jockey             0.0054      0.003      2.020      0.043       0.000       0.011
wins_percent_last_year_jockey          0.2208      0.217      1.020      0.308      -0.204       0.645
wins_percent_2nd_last_year_jockey      0.2134      0.237      0.900      0.368      -0.251      